## Here I am reading the dataset csv file

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime


    
df = pd.read_csv("C:\\Users\\hp\\Desktop\\Online_Retail_Data_Set.csv",encoding='unicode_escape')
print(df.head())


  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  1/12/2010 8:26       2.55     17850.0  United Kingdom  
1  1/12/2010 8:26       3.39     17850.0  United Kingdom  
2  1/12/2010 8:26       2.75     17850.0  United Kingdom  
3  1/12/2010 8:26       3.39     17850.0  United Kingdom  
4  1/12/2010 8:26       3.39     17850.0  United Kingdom  


## Preprocessing  and creating a list of unique tuples of CustomerID and Country

In [2]:
df = df.drop_duplicates()
df['Description'] = df['Description'].fillna("Unknown")
df= df.dropna()
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df=df[df['UnitPrice'] > 0]
df=df[df['Quantity'] > 0]
df=df[~df['Description'].str.contains('Adjust bad debt',case=False)]
lis1=[]
df['CustomerID']=df['CustomerID'].astype(int)
tuples_list1 = [tuple(row) for index, row in df[['CustomerID','Country']].iterrows()]
lis1=list(set(tuples_list1))
print(lis1[0])

(14423, 'United Kingdom')


## Creating a list of unique tuples of StockCode and Description

In [3]:
lis2=[]
tuples_list2 = [tuple(row) for index, row in df[['StockCode','Description']].iterrows()]
lis2=list(set(tuples_list2))

## Creating a list of unique tuples of InvoiceNo,Quantity,UnitPrice,CustomerID,StockCode,InvoiceDate

In [4]:
lis3=[]
tuples_list3 = [tuple(row) for index, row in df[['InvoiceNo','Quantity','UnitPrice','CustomerID','StockCode','InvoiceDate']].iterrows()]
lis3=list(set(tuples_list3))
print(len(lis3))

392690


## Connecting to postgres database

In [5]:
import psycopg2

hostname='localhost'
username='postgres'
database='postgres'
pwd='abhi0395'
port_id=5432
conn=None
cur=None

## Tried to insert the dataset to the postgres database. The tables are created perfectly with each of its variable and constraits.However getting an error when trying to populate it with information.

In [6]:
try:
    conn=psycopg2.connect(
            host=hostname,
            dbname=database,
            user=username,
            password=pwd,
            port=port_id
    )
    cur = conn.cursor()
    create_script1='''  CREATE TABLE  IF NOT EXISTS Customers(
      Customer_Id INT PRIMARY KEY,
      Country varchar(40)
      );'''
    create_script2='''CREATE TABLE  IF NOT EXISTS Stocks(
      Stock_Id INT PRIMARY KEY,
      Description varchar(40)
      );'''
    create_script3='''CREATE TABLE  IF NOT EXISTS Invoice(
      Invoice_Id INT PRIMARY KEY,
      Quantity INT,
      Unit_Price INT,
      Customer_Id INT,
      Stock_Id INT ,
      Invoice_Date Date,
      FOREIGN KEY(Customer_Id) REFERENCES Customers(Customer_Id),
      FOREIGN KEY(Stock_Id) REFERENCES Stocks(Stock_Id)
      );'''
    #cur.execute(create_script1)
    insert_script1='INSERT INTO Customers(Customer_Id,Country) VALUES(%s,%s)'
    insert_value1=lis1
    i=0
    for obj in insert_value1:
        if i==0:
            i=i+1
            continue
        cur.execute(insert_script1,(obj[0],obj[1]))
    conn.commit()
#     insert_script2='INSERT INTO Stocks(Stock_Id,Description) VALUES(%s,%s)'
    
#     insert_script3='INSERT INTO Invoice(Invoice_Id,Quantity,Unit_Price,Customer_Id,Invoice_Date) VALUES(%s,%s)'
except Exception as error:
    print("Ab")
    print(error)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Ab
duplicate key value violates unique constraint "customers_pkey"
DETAIL:  Key (customer_id)=(12457) already exists.

